## STAT628 Module 3 Yelp Data 
### Brian Tsai 
#### mtsai36@wisc.edu

### Importing Required Packages

In [1]:
import numpy as np
import pandas as pd
import os
import json
from scipy import stats as st 
from collections import Counter
import matplotlib.pyplot as py
import seaborn as sns 
from langdetect import detect_langs
import re

ModuleNotFoundError: No module named 'langdetect'

### Reading Business json data 

In [ ]:
data = []
with open('C:/Users/tsai_/Downloads/yelp_dataset/yelp_dataset/business.json', encoding = 'utf8') as fl:
    for i, line in enumerate(fl):
        data.append(json.loads(line))

# Mexican restaurants 
mex = [] 
for i in range(len(data)): 
    if data[i]['categories'] != None: 
        if 'Mex' in data[i]['categories']:
            mex.append(data[i])

# Mexican restuarants that are open            
mex_open = []
for i in mex: 
    if i['is_open'] == 1: 
        mex_open.append(i)


# Business id for open Mexican restuarants         
mex_id = []
for i in mex_open: 
    mex_id.append(i['business_id'])

# Tacobell restaurants with >= 10 reviews  
tacobell = [] 
tacobell_id = [] 
for i in mex_open: 
    if 'Taco Bell' in i['name'] and i['review_count'] >=10: 
        tacobell.append(i)
        tacobell_id.append(i['business_id'])

    
# Chipotle restaurants with >= 10 reviews 
chipotle = [] 
chipotle_id = [] 
for i in mex_open:
    if 'Chipotle Mex' in i['name'] and i['review_count'] >= 10:
        chipotle.append(i) 
        chipotle_id.append(i['business_id'])


## User json file 

In [ ]:
user = []
with open('C:/Users/tsai_/Downloads/yelp_dataset/yelp_dataset/user.json', encoding = 'utf8') as fl:
    for i, line in enumerate(fl):
        user.append(json.loads(line))
df = pd.DataFrame(user)
print(len(user))
df.head()


## Tips json file 

In [ ]:
tacobell_tips = []
chipotle_tips = []
with open('C:/Users/tsai_/Downloads/yelp_dataset/yelp_dataset/tip.json', encoding = 'utf8') as fl:
    for i, line in enumerate(fl):
        x = json.loads(line)
        if x['business_id'] in tacobell_id: 
            tacobell_tips.append(x)
        if x['business_id'] in chipotle_id: 
            chipotle_tips.append(x)

## Reviews 

In [ ]:
# Tacobell and Chipotle Reviews         
tacobell_reviews = []
chipotle_reviews = [] 
with open('C:/Users/tsai_/Downloads/yelp_dataset/yelp_dataset/review.json', encoding = 'utf8') as fl:
    for i, line in enumerate(fl):
        x = json.loads(line)
        if x['business_id'] in tacobell_id: 
            tacobell_reviews.append(x)
        if x['business_id'] in chipotle_id: 
            chipotle_reviews.append(x)        



## Preliminary Analysis

In [ ]:
# How the stars are distributed in the reviews 
# How the stars are distributed among the restaurants in the chain 


# Overall stars rating statistics
tacobell_stars = []
chipotle_stars = []
for i in tacobell_reviews:
    tacobell_stars.append(i['stars'])
for j in chipotle_reviews:
    chipotle_stars.append(j['stars'])

tacobell_avg_rating = sum(tacobell_stars) / len(tacobell_reviews)
chipotle_avg_rating = sum(chipotle_stars) / len(chipotle_reviews)

tacobell_stars_dist = Counter(tacobell_stars)
chipotle_stars_dist = Counter(chipotle_stars)

# Stars rating distribution for each tacobell restaurant
d1 = {}
for i in tacobell_id:
    d1[i] = Counter([j['stars'] for j in tacobell_reviews if j['business_id'] == i])

# Average stars rating for each tacobell restaurant
def f(i):
    """ Returns avg rating for inputted restaurant id"""
    c = 0
    s = 0
    for r in tacobell_reviews:
        if r['business_id'] == i:
            c += r['stars']
            s += 1
    return c/s 

tacobell_res_avg_rating = {}
for i in tacobell_id:
    tacobell_res_avg_rating[i] = f(i)

# Stars rating distribution for each chipotle restaurant
d2 = {}
for i in chipotle_id:
    d2[i] = Counter([j['stars'] for j in chipotle_reviews if j['business_id'] == i])

# Average stars rating for each tacobell restuarant
def g(i):
    """ Returns avg rating for inputted restaurant id"""
    c = 0
    s = 0
    for r in chipotle_reviews:
        if r['business_id'] == i:
            c += r['stars']
            s += 1
    return c/s 

chipotle_res_avg_rating = {}
for i in chipotle_id:
    chipotle_res_avg_rating[i] = g(i)


## Some Tacobell Plots

In [ ]:
# Number of tacobell reviews per month plot
path = 'C:/Users/tsai_/Desktop/U Wisconsin - Madison/Courses/2nd year/Fall 2021/STAT 628/Module 3'
df = pd.DataFrame(tacobell_reviews)
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')
py.plot(df['text'].resample('M').count())
py.xlabel('Year')
py.ylabel('Number of reviews')
py.title('Number of reviews per month')
py.savefig(path + 'tacobell reviews per month.png')
py.show()
py.close()


# Distribution of Tacobell Customer ratings
ax = sns.barplot(data=df, x='stars', y='stars', estimator=lambda x: len(x) / len(df) * 100)
ax.set(ylabel='Percent')
py.title('Distribution of Customer Rating')
py.savefig(path + 'Tacobell customer ratings distribution.png')
py.show()
py.close()

# Average Monthly Customer Rating Tacobell
py.plot(df['stars'].resample('M').mean())
py.xlabel('Year')
py.ylabel('Rating')
py.title('Average Monthly Customer Rating')
py.ylim(0, 5)
py.savefig(path + 'tacobell average monthly customer rating.png')
py.show()
py.close()



## Preliminary NLP 

In [ ]:
# Checking for reviews not in English 
language = [detect_langs(i) for i in df.text]
languages = [str(i[0]).split(':')[0] for i in language]
df['language'] = languages
# Remove reviews not in English 
df = df[df.language == 'en'] 

